# Welcome to RaBPro!

This example demonstrates some of the core functionality and export features provided by RaBPro.

In [ ]:
import rabpro
import os

First, we need to specify a point within our test DEM region. We can also specify a drainage area, which can be approximated by looking at the test region's MERIT_upa data.

In [ ]:
coords = (56.22659, -130.87974)

da = 1994

In order for the analysis to succeed, we need to add the relevent MERIT data files into rabpro's data directory from the `tests/data/` directory. Each data layer (e.g. `dem`, `fdr`, `uda`, etc.) has a corresponding directory, which can be seen below.

In [ ]:
datapaths = rabpro.utils.get_datapaths()
datapaths

When adding new data tiles to the data directories, you should delete `*.vrt` and `*.txt` files. rabpro will recreate these files with the updated data. If no new data is added, there is no need to delete the virtual raster.

In [ ]:
os.remove(datapaths['DEM'])
os.remove(datapaths['DEM_fdr'])
os.remove(datapaths['DEM_uda'])
os.remove(datapaths['DEM_elev_hp'])
os.remove(datapaths['DEM_width'])

Now we can initialize the profiler. The rabpro profiler is the main entry point into the package - it provides wrapper funcitons for most of the rabpro's core functionality.

Note that provided the drainage area (`da`) is optional. We can also set `force_merit` to `True`, which ensures that we use MERIT data rather than HydroBasins to perform basin delineation.

In [ ]:
rpo = rabpro.profiler(coords, name='basic_test', da=da, force_merit=True)

rabpro can now compute the watershed for this point. This can take a few minutes to run.

Requires `test/dem.tif`, `test/fdr.tf`, `test/uda.tif`, and `test/wth.tif`.

In [ ]:
rpo.delineate_basins()

Next, we compute the river elevation profile.

Requires `test/dem.tif`, `test/fdr.tf`, `test/uda.tif`, and `test/wth.tif`.

In [ ]:
rpo.elev_profile()

The basin geometry is stored in a GeoPandas GeoDataFrame, and can be accessed through the `rpo` object. We can also export it as a file using the GeoPandas API.

In [ ]:
basins = rpo.basins

rpo.basins.to_file(rpo.paths['subbasins'], driver='GeoJSON')

Alternatively, you can use `rabpro.profiler`'s `export()` functionality to export elevation and subbasn data as GeoJSON files.

In [ ]:
rpo.export("all")

Once the subbasins are delinated, rabpro can use Google Earth Engine (GEE) to compute statistics for each subbasin. Using Google Earth Engine reduces the need to store large datasets locally, and speeds up computation by using GEE's parallel distributed computing capabilities.

Note: In order to use rabpro for basin statistics, you'll need to sign up for a GEE account. See rabpro's documentation for more information.

In [ ]:
# Specify which statistics to calculate for the JRC/GSW1_3/GlobalSurfaceWater dataset's occurrence band
statlist = ['min', 'max', 'range', 'std', 'sum', 'pct50', 'pct3']
data = rabpro.subbasin_stats.Dataset("JRC/GSW1_3/GlobalSurfaceWater", "occurrence", stats=statlist)

d, t = rpo.basin_stats([data], folder="rabpro test")

The output data will be placed in the `rabpro test` folder in your Google Drive if it already exists. If not, GEE will create a new `rabpro test` folder at the root level of your Drive.